In [ ]:
!pip install -q google-adk

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set


# Agent B: Customer Support Bot (Google ADK)

This exercise rebuilds the customer support agent using the **Google Agent Development Kit (ADK)**.

Compare this with exercise 07 — notice how much boilerplate disappears:
- No manual message history management
- No tool schema definitions (inferred from type hints + docstrings)
- No agentic loop
- No dispatcher function

The ADK handles all of that for you.

## Step 1: Setup and fake database

In [1]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.genai import types

ORDERS = {
    "1": {"item": "Wireless Headphones", "status": "shipped", "tracking": "TRK-98765", "date": "2025-02-20", "price": 79.99},
    "2": {"item": "USB-C Hub", "status": "delivered", "tracking": "TRK-11223", "date": "2025-02-10", "price": 34.99},
    "3": {"item": "Mechanical Keyboard", "status": "processing", "tracking": None, "date": "2025-02-27", "price": 149.99},
}

RETURN_POLICY = """
Items can be returned within 30 days of delivery for a full refund.
Items must be in original packaging and unused condition.
Electronics with opened packaging can only be exchanged, not refunded.
Shipping costs for returns are covered by the customer unless the item is defective.
"""

## Step 2: Define tools as plain functions

The ADK infers the tool schema from:
- **Type hints** → parameter types
- **Docstring** → tool description

No decorators needed — just plain Python functions!

In [2]:
def lookup_order(order_id: str) -> str:
    """Look up an order by its order ID. Returns order details including item, status, tracking, and price."""
    order = ORDERS.get(order_id)
    if order:
        return str(order)
    return f"No order found with ID {order_id}"


def get_return_policy() -> str:
    """Get the store's return and refund policy."""
    return RETURN_POLICY


def check_return_eligibility(order_id: str) -> str:
    """Check if a specific order is eligible for return based on delivery date and status."""
    order = ORDERS.get(order_id)
    if not order:
        return f"No order found with ID {order_id}"
    if order["status"] != "delivered":
        return f"Order {order_id} is not yet delivered (status: {order['status']}). Returns can only be initiated for delivered orders."
    return f"Order {order_id} ({order['item']}) was delivered on {order['date']}. It IS eligible for return within 30 days."

## Step 3: Create the agent

The `Agent` class replaces the system prompt + tools + loop pattern. Everything is declared in one place.

In [3]:
agent = Agent(
    model="gemini-3-flash-preview",
    name="techshop_support",
    instruction="""You are a friendly customer support agent for TechShop, an online electronics store.

Guidelines:
- Be empathetic and helpful
- Always look up order details before answering order questions — never guess
- If an order is not found, apologize and ask the customer to double-check
- When discussing returns, always check the return policy first
- Keep responses concise but warm""",
    tools=[lookup_order, get_return_policy, check_return_eligibility],
)

## Step 4: Run the agent

`InMemoryRunner` + `run_async()` handles the entire agentic loop — calling the model, executing tools, and looping until done.

Note: We use `await` directly because Jupyter already runs an event loop.

In [ ]:
runner = InMemoryRunner(agent=agent, app_name="techshop_support")
runner.auto_create_session = True
user_content = types.Content(role="user", parts=[types.Part.from_text(text="I want to return order 2. Is that possible?")])

final_text = ""
async for event in runner.run_async(user_id="user1", session_id="session1", new_message=user_content):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.text:
                final_text = part.text

print(final_text)

## Try it yourself

Change the input message. Some ideas:
- "Where is my order 1?" — triggers order lookup + tracking info
- "Can I return order 3?" — should say it's not delivered yet
- "What's your return policy?" — just fetches the policy document
- "I want to return order 99" — tests the not-found case

For an interactive chat version:
```bash
uv run python foundation/gemini/08_agent_support/agent_support.py
```

## What the ADK eliminated

Compare this with exercise 07's manual approach:

| Manual (exercises 01-07) | Google ADK (this exercise) |
|---|---|
| Python function stubs for schema | Plain functions with docstrings |
| `contents` list management | Handled by `InMemoryRunner` |
| `while True` agentic loop | `run_async()` |
| `call_tool()` dispatcher | ADK calls functions directly |
| Manual `dict(fc.args)` handling | ADK parses args automatically |

The ADK removes the verbosity so you can focus on what matters: the agent's instructions and tools.